In [92]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

#print('key:', OPENAI_API_KEY)

In [93]:
from openai import OpenAI

client = OpenAI(api_key = OPENAI_API_KEY)

response = client.responses.create(
    model='gpt-4.1',
    input='어벤져스 시리즈의 토르의 역할에 대해 한줄로 설명해줘'
)

In [94]:
response.output_text

'토르는 북유럽 신화의 천둥의 신으로, 어벤져스 시리즈에서 강력한 힘과 묠니르 망치를 사용해 지구와 우주를 위협으로부터 지키는 슈퍼히어로 역할을 한다.'

In [95]:
response = client.responses.create(
    model='gpt-4.1',
    instructions='당신은 영화평론가야',
    input="어벤져스 시리즈의 토르의 역할에 대해 한줄로 설명해줘"
)
print(response.output_text)

토르는 신의 힘과 인간적인 유머를 겸비해 팀에 강력한 전투력과 따뜻한 매력을 더하는 어벤져스의 핵심 영웅입니다.


In [96]:
response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role' : 'developer',
            'content' : '게임에 대한 이야기 하는 것을 좋아해'
        },
        {
            'role' : 'user', 
            'content' : '오늘은 무슨 게임을 할까?'
        }]
)
print(response.output_text)

너무 좋은 질문이야! 어떤 종류의 게임을 하고 싶어?  
1. **친구들이랑 같이 할 멀티플레이 게임**  
2. **이야기나 세계관이 깊은 싱글 플레이 게임**  
3. **빠르고 가벼운 캐주얼 게임**  
4. **도전 욕구를 자극하는 경쟁적인 게임**  
5. **최근 인기 많은 신작 게임**  

조금 더 구체적으로 네가 좋아하는 장르(예: RPG, 전략, FPS, 퍼즐, 시뮬레이션 등)나 가지고 있는 게임기(PC, 콘솔, 모바일, 스위치 등)도 알려주면 딱 맞는 게임을 추천해줄 수 있어!  
어떤 게임 스타일이 좋아? 아니면, 오늘 특별히 끌리는 무드(공포, 힐링, 액션, 협동 등)가 있을까?


In [97]:
completion = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {
            'role' : 'user',
            'content' : '어벤져스 시리즈에 대해 한 문장으로 설명해줘'
        }]
)


In [98]:
#dict(completion.choices[0])
completion.choices[0].message.content

'어벤져스 시리즈는 마블 코믹스의 슈퍼히어로들이 힘을 합쳐 지구를 위협하는 강력한 악당들에 맞서는 이야기를 그린 영화 시리즈입니다.'

In [100]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

user_input = input("질문을 입력해주세요 : ")
prompt = input("프롬프트를 입력해주세요 : ")

response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role' : 'developer',
            "content": prompt
    
        },
        {
            'role' : 'user',
            "content" : user_input 
            
        }]
)
print(response.output_text)

안녕하세요! 만나서 반가워요.  
오늘은 무엇을 배우고 싶으신가요? 궁금한 것이나 도움이 필요한 것이 있으면 언제든 말씀해 주세요. 😊


File Search

In [1]:
import requests
from io import BytesIO
from openai import OpenAI
client = OpenAI()
def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
    # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
    )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
            file=file_content,
            purpose="assistants"
)
    print(result.id)
    return result.id


In [5]:
file_id = create_file(client, './howto-sockets.pdf')

file-Jdv4194zqQmV4We2X77koU


In [6]:
vector_store = client.vector_stores.create(
    name='knowledge_base'
)

print(vector_store.id)

vs_685cb6bce2dc8191bf3c5941f5d822e2


In [ ]:
client.vector_stores.files.create(
    vector_store_id=vector_store.id
    file_id=file_id
)